# FORESEE - Massive spin-2 mediator

In [ ]:
import numpy as np
import sys
import os

src_path = "../../src"
sys.path.append(src_path)
from foresee import Foresee, Utility, Model

from main import sigma_gNucleus_G2Nucleus_analyt, sigma_G2Nucleus_gNucleus_analyt, sigma_ge_G2e_analyt, sigma_G2e_ge_analyt
from constants import *


from matplotlib import pyplot as plt
import matplotlib.tri as tri

plt.rc('text', usetex=True)
plt.rcParams['figure.dpi'] = 400

plt.rcParams['text.latex.preamble'] = [r"\usepackage{amsmath}"]
plt.rcParams['text.latex.preamble'] = [r"\usepackage{amssymb}"]
plt.rcParams['text.latex.preamble'] = [r"\usepackage{siunitx}"]
font = {'family': 'serif', 'serif': ['computer modern roman']}

plt.rc('font', **font)

SMALL_SIZE = 10
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=MEDIUM_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

## 1. Initialization 

In [ ]:
foresee = Foresee()

energy = "0.12"
modelname = "Spin2_massive"
model = Model(modelname)

In [ ]:
num_of_masses = 51
masses = [float(x) for x in ['{:0.4e}'.format(m) for m in np.logspace(-2,0,21+1)]]
masses = np.array(masses)

nsample = 100

#### Add mesons decays

In [ ]:
# # V(p0) -> gamma(p1) + G2(p2)
# # p1**2 = m1**2 = 0
# # p2**2 = m2**2 = m_G2**2

# model.add_production_2bodydecay(
#     pid0 = "113", # rho
#     pid1 = "0",   # pid1 = 0 means mass_pid1 = m1 = mass_llp0;   mass_pid2 = m2 = mass = mass_llp1
#     br = "4.72e-5 * coupling**2 * (1 * (m0**2 - m2**2)**3) / (8. * ALPHAEM * m0 * np.sqrt(m0**2 - 4*M_ELECTRON**2) * (m0**2 + 2*M_ELECTRON**2) * np.pi)",
#     generator = "SeaQuest",
#     energy = energy,
#     nsample = nsample,
# )

# model.add_production_2bodydecay(
#    pid0 = "223", # omega
#    pid1 = "0",
#    br = "7.38e-5 * coupling**2 * (1 * (m0**2 - m2**2)**3) / (8. * ALPHAEM * m0 * np.sqrt(m0**2 - 4*M_ELECTRON**2) * (m0**2 + 2*M_ELECTRON**2) * np.pi)",
#    generator = "SeaQuest",
#    energy = energy,
#    nsample = nsample,
# )

# model.add_production_2bodydecay(
#    pid0 = "333", # phi
#    pid1 = "0",
#    br = "2.98e-4 * coupling**2 * (1 * (m0**2 - m2**2)**3) / (8. * ALPHAEM * m0 * np.sqrt(m0**2 - 4*M_ELECTRON**2) * (m0**2 + 2*M_ELECTRON**2) * np.pi)",
#    generator = "SeaQuest",
#    energy = energy,
#    nsample = nsample,
# )

# model.add_production_2bodydecay(
#     pid0 = "443", # J/ψ
#     pid1 = "0",
#     br = "0.0597 * coupling**2 * (1 * (m0**2 - m2**2)**3) / (8. * ALPHAEM * m0 * np.sqrt(m0**2 - 4*M_ELECTRON**2) * (m0**2 + 2*M_ELECTRON**2) * np.pi)",
#     generator = "SeaQuest",
#     energy = energy,
#     nsample = nsample,
# )


In [ ]:
model.add_production_direct(
    label = "SeaQuest_Prim",
    energy = energy,
    coupling_ref=1,
    masses=masses,
)

In [ ]:
model.set_ctau_1d(filename="model/ctau_G2.txt", coupling_ref=1)

branchings = [
    [ "BR_G2_gg", "black", "solid", r"$\gamma\gamma$", 0.110, 0.30 ],  
    [ "BR_G2_ee", "red", "solid",  r"$e^+ e^-$", 0.110, 0.016],
    [ "BR_G2_mumu", "green", "solid",  r"$\mu^+ \mu^-$", 0.210, 0.016],
]

model.set_br_1d(
    modes=[channel for channel, _, _, _, _, _ in branchings],
    filenames=[ "model/br/" + channel + ".txt" for channel, _, _, _, _, _ in branchings ], )

foresee.set_model(model=model)

In [ ]:
# %matplotlib inline

# mass_llp0 = 0.0 # m_gamma
# mass_llp1 = 0.1 # m_G2

# plt_2 = foresee.get_llp_spectrum(mass=mass_llp1, mass_llp0=mass_llp0, coupling=1e-4, do_plot=True, save_file=False, is_llp0_spectrum_zero=True)
# plt_2.savefig("./output/test_LLP_spect_plt_2.pdf")
# plt_2.show()


# plt_2 = foresee.get_llp_spectrum(mass=mass_llp1, mass_llp0=mass_llp0, coupling=1e-4, do_plot=True, save_file=False, detector="MATHUSLA", is_llp0_spectrum_zero=True)
# plt_2.savefig("./output/test_LLP_spect_plt_2_MATHUSLA.pdf")
# plt_2.show()

In [ ]:
from timeit import default_timer as timer


for count, mass in enumerate(masses):
    mass_llp0 = 0.0    # photon
    mass_llp1 = mass   # G2

    start = timer()
    
    foresee.get_llp_spectrum(mass=mass_llp1, mass_llp0=mass_llp0, coupling=1, detector="SeaQuest", stat_cuts_llp0="p_llp0.e>0.0", stat_cuts_llp1="p_llp1.e>0.0")

    end = timer()
    time_length_sec = end - start

    count += 1
    time_length_sec_total = time_length_sec * num_of_masses
    print("%.2f%% done, " % float(count / num_of_masses * 100), "approx. total run time : %.1f m, " % float(time_length_sec_total / 60), "approx. waiting time: %.1f m" % float(time_length_sec_total * (1 - count / num_of_masses) / 60))

In [ ]:
# productions = [
#     ["SeaQuest_Prim"    , None     , "green"    ,    r"Prim."         ],   
#     # ["111"    , None      , "firebrick"   , r"$\pi$"         ],   
#     # ["221"    , None      , "red"         , r"$\eta$"        ],   
#     # ["331"    , None , "salmon"      , r"$\eta'$"       ],  
#     ["113"    , None , "dodgerblue"  , r"$\rho$"        ],   
#     ["223"    , None , "blue"        , r"$\omega$"      ],   
#     ["333"    , None , "deepskyblue" , r"$\phi$"        ],  
#     ["443"    , None  , "gold"        , r"$J/\psi$"      ],   
#     # ["100443" , None  , "orange"      , r"$\psi(2S)$"    ],  
#     # ["553"    , None  , "green"       , r"$\Upsilon(1S)$"],   
#     # ["100553" , None  , "limegreen"   , r"$\Upsilon(2S)$"],  
#     # ["200553" , None  , "lime"        , r"$\Upsilon(3S)$"],  
# ]

# plot = foresee.plot_production(
#     , 
#     productions = productions,
#     condition="True", 
#     xlims=[0.03,0.3],
#     ylims=[1e15,1e20], 
#     xlabel=r"$m_G$ [GeV]", 
#     ylabel=r"Production rate $\sigma \times g_\gamma^2$ [a.u.]",
#     legendloc=(1.02,1.02),
#     fs_label=12,
#     dolegend=True,
#     energy="0.12",
#     detector="SeaQuest",
# )

# plot.subplots_adjust(left=0.11, right=0.99, bottom=0.10, top=0.97)
# plot.savefig("output/Production_channels_SeaQuest.pdf")
# plot.show()

## 5. Parameter Scan

In [ ]:
num_of_couplings = 99

luminosity, distance = 1/1000, 52.7
setup, selection, channels, length = "SeaQuest_gg", "np.sqrt(x.x**2 + x.y**2) >= 0", [ "BR_G2_gg" ], 50
foresee.set_detector(length=length, selection=selection, channels=channels, distance=distance, luminosity=luminosity)

list_nevents = []    
for mass in masses:
    couplings, _, nevents, _, _  = foresee.get_events(mass=mass, energy=energy, couplings=np.logspace(-8, -2, num_of_couplings), detector="SeaQuest", preselectioncuts="p>3")
    list_nevents.append(nevents)  
        
#save results
np.save("model/results/"+energy+"TeV_"+setup+".npy",[masses,couplings,list_nevents])

In [ ]:
luminosity, distance = 1/1000, 52.7
setup, selection, channels, length = "SeaQuest_ee", "np.sqrt(x.x**2 + x.y**2) >= 0", [ "BR_G2_ee" ], 50
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)

list_nevents = []
for mass in masses:
    couplings, _, nevents, _, _ = foresee.get_events(mass=mass, energy=energy, couplings=np.logspace(-8, -2, num_of_couplings), detector="SeaQuest", preselectioncuts="p>3")
    list_nevents.append(nevents)
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])

In [ ]:
luminosity, distance = 1/1000, 52.7
setup, selection, channels, length = "SeaQuest_mumu", "np.sqrt(x.x**2 + x.y**2) >= 0", [ "BR_G2_mumu" ], 50
foresee.set_detector(length=length,
                     selection=selection,
                     channels=channels,
                     distance=distance,
                     luminosity=luminosity)

list_nevents = []
for mass in masses:
    couplings, _, nevents, _, _ = foresee.get_events(mass=mass, energy=energy, couplings=np.logspace(-8, -2, num_of_couplings), detector="SeaQuest", preselectioncuts="p>3")
    list_nevents.append(nevents)
np.save("model/results/" + energy + "TeV_" + setup + ".npy", [masses, couplings, list_nevents])